# Thatcher Simulation Model 1.0

In this document, we detail a simulation of the most basic model of monitoring in auto-insurance with adverse selection (only)

In [2]:
#Create hash table for premiums
source("./code/pricing_contractchoice_dev.R")

[1] "Set the wd to : /Volumes/Transcend/Dropbox/Project_Thatcher/Simulations"


Saving 7 x 7 in image


## Base Model Description
We begin with the simplest dynamic model of insurance plan choice s.t.:
- Accident risk is inherent and static across time
- All agents have the same coefficient of absolute risk aversion

## Building Out the Model

Each individual is endowed with a private risk of accidents, $\lambda$.

We assume that the (total) number of accidents $NumA_t$ is distributed according to $Poisson(\lambda)$.

Each accident that occurs is independently chosen to be either a major accident (with Bernoulli probability $p_{Maj}$) or a minor accident (with probability $1-p_{Maj}$).

For now, we say $p_{Maj} \sim U[0,.2]$ and is privately known.

In [3]:
# Start out with 1 driver with uniform probs:
lambda_i = runif(1,max=.3)
pMaj_i = runif(1,max=.2)

We assume that the cost of each accident is an iid lognormal distribution:
$$C_{Maj} \sim log\mathcal{N}(\mu_{Maj},1)$$
$$C_{Min} \sim log\mathcal{N}(\mu_{Min},1)$$

For starters, we assume $mu_{Maj} = 500$ and $mu_{Min} = 7$

In [ ]:
#To differentiate major and minor accident costs, make up some means here:
meanMj = 500
meanMn = 7

We assume that all agents have a form of CARA utility:
$$u(c) = c - \alpha c^2 + \xi $$ where $\xi \sim \mathcal{N}(0,\sigma^2)$ is a an instance-specific random error.

Initially, we assume that all agents have the same CARA coefficient: $\alpha \approx \frac{0.05}{1-2000*0.05}.$

In [4]:
# Initially assume all drivers have the same risk aversion parameter;
# expand this later
gamma_ra = .05
alpha_ra = -gamma_ra/2000

# Risk-averse Utility function
getUtility <- function(alpha=alpha_ra,cost){
  utility = -cost - alpha*cost^2
  return(utility)
}

An agent's flow utility is determined by his expected utility:
$$E = \sum_{m,M} \int_{c_m>l_y,c_M>l_y} u(\pi(y,m,M)) dF(c_m,c_M) Pr(m,M)$$
where $\pi(y,m,M) = - p(y) - OOP(m,M)$ is the total payment.

We implement this with a monte carlo integration:

In [3]:
#
numAccidentsRV <- function(numDraws=1,lam){
  return(rpois(numDraws,lam))
}

# Wrapper function for random draws of the total costs of accidents (minus deductible)
getTotalCostOfAccidentsMinusDeductibleRV <- function(numDraws=1,mean_cost,u_l_var){
  if(numDraws>0){
  # Current Cost Distribution: Log-normal rv with mean, mean_cost, and std dev = 100
  costDraws = rlnorm(n = numDraws,meanlog = log(mean_cost),sdlog = 1) - rep(u_l_var,numDraws)
  posCostDraws = pmax(costDraws,rep(0,numDraws))
  totalCost = sum(posCostDraws)
  return(totalCost)
  }
  else{
    return(0)
  }
}return(totalCost)
}

vectorized_getTotalCostOfAccidentsRV <- Vectorize(getTotalCostOfAccidentsMinusDeductibleRV)



# Archived Code